In [4]:
import pandas as pd

### Basic example: Error Type 1

In [17]:
df = pd.read_excel(
    '../Data/TidyData.xlsx',
    sheet_name='Bad_Example1'
)

#####  Mit Stack

In [18]:
df

,Unnamed: 0,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [19]:
df = df.rename(columns={'Unnamed: 0': 'names'})

In [20]:
df.set_index('names').stack()

names                   
John Smith    treatmentb     2.0
Jane Doe      treatmenta    16.0
              treatmentb    11.0
Mary Johnson  treatmenta     3.0
              treatmentb     1.0
dtype: float64

##### Mit melt

In [21]:

value_columns=['treatmenta', 'treatmentb']
df = df.melt(id_vars='names', value_vars=value_columns)

In [22]:
df.variable = df.variable.str[-1]

In [23]:
df.rename(columns={'variable': 'treatment', 'value': 'result'})

,names,treatment,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


### Basic example: Error Type 2

In [8]:
df = pd.read_excel('../Data/TidyData_ErrorType_Two.xlsx')


In [9]:

df = df.pivot(
    index=['machine', 'date'],
    columns=['key'],
    values='value'
)

In [10]:
df = df.reset_index()

In [11]:
df.columns.name = ''

In [12]:
df

,machine,date,t_max,t_min
0,M,2022-01-01,17.0,15.0
1,M,2022-01-02,17.4,15.2
2,M,2022-01-03,18.3,15.9


### Pew

In [7]:
df = pd.read_excel('pew.xlsx')

In [8]:
df = df.rename(columns={'Unnamed: 0': 'religion'})

In [12]:
value_columns = [x for x in df.columns if 'religion' not in x]
# x=df.columns.to_list()

# df = df.melt(id_vars='religion', value_vars=value_columns)

In [25]:
df

,religion,variable,value
0,Buddhist,"$150,000 or more",53
1,Catholic,"$150,000 or more",633
2,Don’t know/refused (no information on religio...,"$150,000 or more",18
3,Evangelical Protestant Churches,"$150,000 or more",414
4,Hindu,"$150,000 or more",54
...,...,...,...
155,Orthodox,"Less than $10,000",13
156,Other Christian,"Less than $10,000",9
157,Other Faiths,"Less than $10,000",20
158,Other World Religions,"Less than $10,000",5


### Tuberculosis

In [13]:
df = pd.read_csv('tb.csv')

In [15]:
df = df[df.year==2000]
df.shape

(201, 23)

In [16]:
df = df.dropna(axis=1, how="all")
df = df.drop(columns='new_sp')


In [17]:
df.columns = [x.strip('new_sp') for x in df.columns.to_list()]
df = df.rename(columns={'iso2': 'country'})


In [18]:
value_columns = [
    x for x in df.columns.to_list() if 'country' not in x and 'year' not in x
]

In [19]:
melted_df = df.melt(
    id_vars=['country', 'year'],
    value_vars=value_columns
)


In [20]:
melted_df['gender'] = melted_df['variable']
melted_df['gender'] = melted_df['gender'].str[0]


In [21]:
melted_df = melted_df.rename(columns={'variable': 'age_range'})
melted_df.age_range = melted_df.age_range.str[1:]


In [22]:
melted_df['min_age'] = melted_df['age_range']
melted_df = melted_df.rename(columns={'age_range': 'max_age'})


In [23]:
from numpy import nan
min_age_func = lambda x: x[0] if len(x)==3 else x[:2]
max_age_func = lambda x: x[-2:] if len(x)>2 else nan

In [24]:
melted_df['max_age'] = melted_df['max_age'].apply(max_age_func)
melted_df['min_age'] = melted_df['min_age'].apply(min_age_func)

In [25]:
melted_df

,country,year,max_age,value,gender,min_age
0,AD,2000,14,0.0,m,0
1,AE,2000,14,2.0,m,0
2,AF,2000,14,52.0,m,0
3,AG,2000,14,0.0,m,0
4,AL,2000,14,2.0,m,0
...,...,...,...,...,...,...
2809,YE,2000,NaN,92.0,f,65
2810,YU,2000,NaN,NaN,f,65
2811,ZA,2000,NaN,80.0,f,65
2812,ZM,2000,NaN,75.0,f,65


### Weather

In [1]:
import pandas as pd

In [2]:
colspecs = [(0,11), (11, 15), (15, 17), (17, 21)]
colspecs = colspecs + [(21 + 8*x, 21 + (8 * x)+5) for x in range(0, 31)]

In [3]:
df = pd.read_fwf(
    '../Data/weather.txt',
    colspecs=colspecs,
    header=None
)

In [4]:
columns = [
    'id',
    'year',
    'month',
    'measurement'
]
days = [x+1 for x in range(0,31)]
columns = columns + days
df.columns = columns

In [5]:
melted_df = df.melt(
    id_vars=[
        'id',
        'year',
        'month',
        'measurement'
    ],
    value_vars=days,
    var_name='day')

In [6]:
melted_df

,id,year,month,measurement,day,value
0,MX000017004,1955,4,TMAX,1,310
1,MX000017004,1955,4,TMIN,1,150
2,MX000017004,1955,4,PRCP,1,0
3,MX000017004,1955,5,TMAX,1,310
4,MX000017004,1955,5,TMIN,1,200
...,...,...,...,...,...,...
53129,MX000017004,2011,3,TMIN,31,170
53130,MX000017004,2011,3,PRCP,31,0
53131,MX000017004,2011,4,TMAX,31,-9999
53132,MX000017004,2011,4,TMIN,31,-9999


In [34]:
melted_df.pivot(
    index=['id', 'year', 'month', 'day'],
    columns=['measurement'],
    values='value'
).reset_index()

measurement,id,year,month,day,PRCP,TMAX,TMIN
0,MX000017004,1955,4,1,0.0,310.0,150.0
1,MX000017004,1955,4,2,0.0,310.0,150.0
2,MX000017004,1955,4,3,0.0,310.0,160.0
3,MX000017004,1955,4,4,0.0,320.0,150.0
4,MX000017004,1955,4,5,0.0,330.0,160.0
...,...,...,...,...,...,...,...
18192,MX000017004,2011,4,27,-9999.0,-9999.0,-9999.0
18193,MX000017004,2011,4,28,-9999.0,-9999.0,-9999.0
18194,MX000017004,2011,4,29,-9999.0,-9999.0,-9999.0
18195,MX000017004,2011,4,30,-9999.0,-9999.0,-9999.0
